In [5]:
import csv as csv
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn_pandas import DataFrameMapper

In [6]:
df_train = pd.read_csv('train.csv', header=0, index_col='PassengerId')
df_test = pd.read_csv('test.csv', header=0, index_col='PassengerId')
# concatenate the data
df = pd.concat([df_train, df_test], keys=["train", "test"])

In [7]:
df

Age        Cabin Embarked      Fare  \
      PassengerId                                         
train 1            22.0          NaN        S    7.2500   
      2            38.0          C85        C   71.2833   
      3            26.0          NaN        S    7.9250   
      4            35.0         C123        S   53.1000   
      5            35.0          NaN        S    8.0500   
      6             NaN          NaN        Q    8.4583   
      7            54.0          E46        S   51.8625   
      8             2.0          NaN        S   21.0750   
      9            27.0          NaN        S   11.1333   
      10           14.0          NaN        C   30.0708   
      11            4.0           G6        S   16.7000   
      12           58.0         C103        S   26.5500   
      13           20.0          NaN        S    8.0500   
      14           39.0          NaN        S   31.2750   
      15           14.0          NaN        S    7.8542   
      16           55.0          NaN        S   16.0000   
      17            2.0          NaN        Q   29.1250   
      18            NaN          NaN        S   13.0000   
      19           31.0          NaN        S   18.0000   
      20            NaN          NaN        C    7.2250   
      21           35.0          NaN        S   26.0000   
      22           34.0          D56        S   13.0000   
      23           15.0          NaN        Q    8.0292   
      24           28.0           A6        S   35.5000   
      25            8.0          NaN        S   21.0750   
      26           38.0          NaN        S   31.3875   
      27            NaN          NaN        C    7.2250   
      28           19.0  C23 C25 C27        S  263.0000   
      29            NaN          NaN        Q    7.8792   
      30            NaN          NaN        S    7.8958   
...                 ...          ...      ...       ...   
test  1280         21.0          NaN        Q    7.7500   
      1281          6.0          NaN        S   21.0750   
      1282         23.0          B24        S   93.5000   
      1283         51.0          D28        S   39.4000   
      1284         13.0          NaN        S   20.2500   
      1285         47.0          NaN        S   10.5000   
      1286         29.0          NaN        S   22.0250   
      1287         18.0          C31        S   60.0000   
      1288         24.0          NaN        Q    7.2500   
      1289         48.0          B41        C   79.2000   
      1290         22.0          NaN        S    7.7750   
      1291         31.0          NaN        Q    7.7333   
      1292         30.0           C7        S  164.8667   
      1293         38.0          NaN        S   21.0000   
      1294         22.0          NaN        C   59.4000   
      1295         17.0          NaN        S   47.1000   
      1296         43.0          D40        C   27.7208   
      1297         20.0          D38        C   13.8625   
      1298         23.0          NaN        S   10.5000   
      1299         50.0          C80        C  211.5000   
      1300          NaN          NaN        Q    7.7208   
      1301          3.0          NaN        S   13.7750   
      1302          NaN          NaN        Q    7.7500   
      1303         37.0          C78        Q   90.0000   
      1304         28.0          NaN        S    7.7750   
      1305          NaN          NaN        S    8.0500   
      1306         39.0         C105        C  108.9000   
      1307         38.5          NaN        S    7.2500   
      1308          NaN          NaN        S    8.0500   
      1309          NaN          NaN        C   22.3583   

                                                                Name  Parch  \
      PassengerId                                                             
train 1                                      Braund, Mr. Owen Harris      0   
      2            Cumings, Mrs. John Bradley (Florence Briggs Th

In [8]:
# new features from names
df['Title'] = df['Name'].apply(lambda c: c[c.index(',') + 2 : c.index('.')])
df['LastName'] = df['Name'].apply(lambda n: n[0:n.index(',')])
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
# replace missing data with the mode of the column
df.loc[df['Embarked'].isnull(), 'Embarked'] = df['Embarked'].mode()[0]
df.loc[df['Fare'].isnull(), 'Fare'] = df['Fare'].mode()[0]
# features for family
df['FamilyID'] = df['LastName'] + ':' + df['FamilySize'].apply(str)
df.loc[df['FamilySize'] <= 2, 'FamilyID'] = 'Small_Family'
# replace missing age data with median by title
df['AgeOriginallyNaN'] = df['Age'].isnull().astype(int)
medians_by_title = pd.DataFrame(df.groupby('Title')['Age'].median()) \
  .rename(columns = {'Age': 'AgeFilledMedianByTitle'})
df = df.merge(medians_by_title, left_on = 'Title', right_index = True) \
  .sort_index(level = 0).sort_index(level = 1)

df

Age        Cabin Embarked      Fare  \
      PassengerId                                         
train 1            22.0          NaN        S    7.2500   
      2            38.0          C85        C   71.2833   
      3            26.0          NaN        S    7.9250   
      4            35.0         C123        S   53.1000   
      5            35.0          NaN        S    8.0500   
      6             NaN          NaN        Q    8.4583   
      7            54.0          E46        S   51.8625   
      8             2.0          NaN        S   21.0750   
      9            27.0          NaN        S   11.1333   
      10           14.0          NaN        C   30.0708   
      11            4.0           G6        S   16.7000   
      12           58.0         C103        S   26.5500   
      13           20.0          NaN        S    8.0500   
      14           39.0          NaN        S   31.2750   
      15           14.0          NaN        S    7.8542   
      16           55.0          NaN        S   16.0000   
      17            2.0          NaN        Q   29.1250   
      18            NaN          NaN        S   13.0000   
      19           31.0          NaN        S   18.0000   
      20            NaN          NaN        C    7.2250   
      21           35.0          NaN        S   26.0000   
      22           34.0          D56        S   13.0000   
      23           15.0          NaN        Q    8.0292   
      24           28.0           A6        S   35.5000   
      25            8.0          NaN        S   21.0750   
      26           38.0          NaN        S   31.3875   
      27            NaN          NaN        C    7.2250   
      28           19.0  C23 C25 C27        S  263.0000   
      29            NaN          NaN        Q    7.8792   
      30            NaN          NaN        S    7.8958   
...                 ...          ...      ...       ...   
test  1280         21.0          NaN        Q    7.7500   
      1281          6.0          NaN        S   21.0750   
      1282         23.0          B24        S   93.5000   
      1283         51.0          D28        S   39.4000   
      1284         13.0          NaN        S   20.2500   
      1285         47.0          NaN        S   10.5000   
      1286         29.0          NaN        S   22.0250   
      1287         18.0          C31        S   60.0000   
      1288         24.0          NaN        Q    7.2500   
      1289         48.0          B41        C   79.2000   
      1290         22.0          NaN        S    7.7750   
      1291         31.0          NaN        Q    7.7333   
      1292         30.0           C7        S  164.8667   
      1293         38.0          NaN        S   21.0000   
      1294         22.0          NaN        C   59.4000   
      1295         17.0          NaN        S   47.1000   
      1296         43.0          D40        C   27.7208   
      1297         20.0          D38        C   13.8625   
      1298         23.0          NaN        S   10.5000   
      1299         50.0          C80        C  211.5000   
      1300          NaN          NaN        Q    7.7208   
      1301          3.0          NaN        S   13.7750   
      1302          NaN          NaN        Q    7.7500   
      1303         37.0          C78        Q   90.0000   
      1304         28.0          NaN        S    7.7750   
      1305          NaN          NaN        S    8.0500   
      1306         39.0         C105        C  108.9000   
      1307         38.5          NaN        S    7.2500   
      1308          NaN          NaN        S    8.0500   
      1309          NaN          NaN        C   22.3583   

                                                                Name  Parch  \
      PassengerId                                                             
train 1                                      Braund, Mr. Owen Harris      0   
      2            Cumings, Mrs. John Bradley (Florence Briggs Th

In [9]:

# reseparate using keys
df_train = df.ix['train']
df_test  = df.ix['test']

In [11]:
df_train

,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Survived,Ticket,Title,LastName,FamilySize,FamilyID,AgeOriginallyNaN,AgeFilledMedianByTitle
PassengerId,,,,,,,,,,,,,,,,,
1,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,0.0,A/5 21171,Mr,Braund,2,Small_Family,0,29.0
2,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,1.0,PC 17599,Mrs,Cumings,2,Small_Family,0,35.5
3,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,female,0,1.0,STON/O2. 3101282,Miss,Heikkinen,1,Small_Family,0,22.0
4,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,female,1,1.0,113803,Mrs,Futrelle,2,Small_Family,0,35.5
5,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,3,male,0,0.0,373450,Mr,Allen,1,Small_Family,0,29.0
6,NaN,NaN,Q,8.4583,"Moran, Mr. James",0,3,male,0,0.0,330877,Mr,Moran,1,Small_Family,1,29.0
7,54.0,E46,S,51.8625,"McCarthy, Mr. Timothy J",0,1,male,0,0.0,17463,Mr,McCarthy,1,Small_Family,0,29.0
8,2.0,NaN,S,21.0750,"Palsson, Master. Gosta Leonard",1,3,male,3,0.0,349909,Master,Palsson,5,Palsson:5,0,4.0
9,27.0,NaN,S,11.1333,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,3,female,0,1.0,347742,Mrs,Johnson,3,Johnson:3,0,35.5


In [13]:
X_train = df_train[df_train.columns.drop('Survived')]
y_train = df_train['Survived']

In [17]:
X_train

,Age,Cabin,Embarked,Fare,Name,Parch,Pclass,Sex,SibSp,Ticket,Title,LastName,FamilySize,FamilyID,AgeOriginallyNaN,AgeFilledMedianByTitle
PassengerId,,,,,,,,,,,,,,,,
1,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,3,male,1,A/5 21171,Mr,Braund,2,Small_Family,0,29.0
2,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,1,female,1,PC 17599,Mrs,Cumings,2,Small_Family,0,35.5
3,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,female,0,STON/O2. 3101282,Miss,Heikkinen,1,Small_Family,0,22.0
4,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,1,female,1,113803,Mrs,Futrelle,2,Small_Family,0,35.5
5,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,3,male,0,373450,Mr,Allen,1,Small_Family,0,29.0
6,NaN,NaN,Q,8.4583,"Moran, Mr. James",0,3,male,0,330877,Mr,Moran,1,Small_Family,1,29.0
7,54.0,E46,S,51.8625,"McCarthy, Mr. Timothy J",0,1,male,0,17463,Mr,McCarthy,1,Small_Family,0,29.0
8,2.0,NaN,S,21.0750,"Palsson, Master. Gosta Leonard",1,3,male,3,349909,Master,Palsson,5,Palsson:5,0,4.0
9,27.0,NaN,S,11.1333,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,3,female,0,347742,Mrs,Johnson,3,Johnson:3,0,35.5
